In [19]:
import pandas as pd
import numpy as np

# read in the data course-v1.csv
df = pd.read_csv('course-v1.csv')

In [20]:
print(df['所屬學院'].size)

8107


In [3]:
# create a new column called embedding text, which is a cancat of # "課程名稱" "課程概述" "課程目標" "課程要求"
df['embedding_text'] = df['課程名稱'] + ' ' + df['課程概述'] + ' ' + df['課程目標'] + ' ' + df['課程要求']

In [4]:
# let all Nan value in all column to be empty string
df = df.fillna('')


In [5]:
# add "passage: " to the start of each row in the embedding_text column
df['embedding_text'] = 'passage: ' + df['embedding_text']

# let the maximum length of the embedding_text column be 500
# df['embedding_text'] = df['embedding_text'].apply(lambda x: str(x)[:8000] if isinstance(x, str) else x)
df['embedding_text'] = df['embedding_text'].apply(lambda x: str(x)[:800] if isinstance(x, str) else x)

In [6]:
# get rid of  df['課程概述'] + ' ' + df['課程目標'] + ' ' + df['課程要求']
df = df.drop(columns=['課程概述', '課程目標', '課程要求'])

In [7]:
# let first 50 courses be a new df, df1
# df1 = df.head(50)
df1 = df


In [8]:
# load environment variables
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.vectorstores import Pinecone
from langchain.embeddings.base import Embeddings
from langchain.schema import Document


In [9]:
# !pip install -qU langchain-openai
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)
# embeddings = PineconeEmbeddings(model="multilingual-e5-large")
index_name = "ntuim-course"


In [10]:
# upload the data to Pinecone, let the matadata be the rows of the df1, and use embedding_text to calculate the embedding

documents = []
# for _, row in df1[:100].iterrows():
for _, row in df1.iterrows():
    # Create the document with text for embeddings and rest as metadata
    metadata = row.drop('embedding_text').to_dict()
    doc = Document(page_content=row['embedding_text'], metadata=row.to_dict())
    documents.append(doc)

# Generate embeddings for the "embedding_text" using the embeddings model
# embeddings = Embeddings()  # Replace with your actual embedding model (BaichuanTextEmbeddings)

In [11]:
docsearch = PineconeVectorStore.from_documents(documents, embedding=embeddings, index_name=index_name)
# upload the embeddings to the Pinecone index